## 机器学习4-神经网络
通过反向传播算法实现神经网络成本函数和梯度计算的非正则化个正则化，还将实现随机权重初始化和使用网络进行预测的方法

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [3]:
data = loadmat('ex4data1.mat')
data['X'].shape,data['y'].shape

((5000, 400), (5000, 1))

In [4]:
X = data['X']
y = data['y']

我们也需要对我们的y标签进行一次one-hot 编码。 one-hot 编码将类标签n（k类）转换为长度为k的向量，其中索引n为“hot”（1），而其余为0。 Scikitlearn有一个内置的实用程序，我们可以使用这个。

In [5]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False,categories='auto')
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(5000, 10)

In [6]:
y[10],y_onehot

(array([10], dtype=uint8), array([[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]]))

### sigmoid 函数(代价函数)
g 代表一个常用的逻辑函数（logistic function）为S形函数（Sigmoid function），公式为： \\[g\left( z \right)=\frac{1}{1+{{e}^{-z}}}\\] 
合起来，我们得到逻辑回归模型的假设函数： 
	\\[{{h}_{\theta }}\left( x \right)=\frac{1}{1+{{e}^{-{{\theta }^{T}}X}}}\\] 

In [23]:
def sigmoid(z):
    return 1 / (1+np.exp(-z))

### Forword Propagation(前向传播)
> (400 + 1) -> (25 + 1) -> (10)

+1是bias units （偏执单元）

<img style="float: left;" src="../img/nn_model.png">

In [24]:
#根据上面的图可以写出下面的公式
def forward_propagate(X,theta1,theta2):
    m = X.shape[0]
    
    #X的第一列插入全为1，即添加偏执单元 bias units
    a1 = np.insert(X,0,values=np.ones(m),axis=1)
    z2 = a1 * theta1.T 
    a2 = np.insert(sigmoid(z2),0,values=np.ones(m),axis=1)
    z3 = a2* theta2.T
    #获取最后的结果
    h = sigmoid(z3)
    
    return a1,z2,a2,z3,h    

###  代价函数
<img style="float: left;" src="../img/nn_cost.png">

In [33]:
# input_size 输入层
# hidden_size 隐藏层，中间层
def cost(params, input_size,hidden_size,num_labels,X,y,learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size +1)],(hidden_size,(input_size + 1 ))))         
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size +1):],(num_labels,(hidden_size + 1 ))))
    
    #前向传播获取各层数据
    a1,z2,a2,z3,h = forward_propagate(X,theta1,theta2)
    
    #计算代价函数
    J = 0
    
    for i in range(m):
        first_term = np.multiply(-y[i,:],np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]) , np.log( 1 - h[i,:]))
        
        J += np.sum(first_term - second_term)
    
    J =  J / m
    
    return J    

In [34]:
#初始化设置
input_size = 400
hidden_size = 25
num_labels = 10 
learningRate = 1

#随机初始化完整网络参数大小的参数数组 
params = (np.random.random(size = hidden_size *(input_size + 1 ) + num_labels *(hidden_size + 1)) -0.5) *0.25      

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

#将参数数组解开为每个层的参数矩阵
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)],(hidden_size,(input_size +1))))       
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):],(num_labels,(hidden_size + 1))))

theta1.shape,theta2.shape



((25, 401), (10, 26))

In [35]:
#计算前向传播
a1,z2,a2,z3,h = forward_propagate(X,theta1,theta2)
a1.shape,z2.shape,a2.shape,z3.shape,h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

In [36]:
cost(params,input_size,hidden_size,num_labels,X,y_onehot,learningRate)

6.793965746160785

### 反向传播算法

In [37]:
# 创建sigmoid函数的梯度函数
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z),(1-sigmoid(z)))

> 现在我们准备好实施反向传播来计算梯度。 由于反向传播所需的计算是代价函数中所需的计算过程，我们实际上将扩展代价函数以执行反向传播并返回代价和梯度。

<img style="float: left;" src="../img/神经网络代价函数.png">

In [52]:
def background_propagation(params ,input_size,hidden_size,num_labels,X,y,learningRateea):  
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size +1)],(hidden_size,(input_size + 1 ))))         
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size +1):],(num_labels,(hidden_size + 1 ))))
    
    #前向传播获取各层数据
    a1,z2,a2,z3,h = forward_propagate(X,theta1,theta2)
    
    #初始化
    J = 0
    delta1 = np.zeros(theta1.shape) #(25,401)
    delta2 = np.zeros(theta2.shape) #(10,26)
    
    for i in range(m):
        first_term = np.multiply(-y[i,:],np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]) , np.log( 1 - h[i,:]))
        
        J += np.sum(first_term - second_term)
    
    J =  J / m
    #添加正则项
    J += (float(learningRate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    #开始反向传播
    for t in range(m):
        a1t = a1[t,:] #（1.401）
        z2t = z2[t,:] #(1,25)
        a2t = a2[t,:] #(1,26)
        ht = h[t,:] #(1,10)
        yt = y[t,:] #(1,10)
        
        #计算预测值和实际值的差值
        d3t = ht - yt 
        
        #z2添加偏执单元 bias units
        z2t = np.insert(z2t,0,values=np.ones(1)) #添加一列值全为1，放在z2的第一列
        d2t = np.multiply((theta2.T * d3t.T).T,sigmoid_gradient(z2t))
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learningRate) / m
    delta2[:,1:] = delta2[:,1:] +(theta2[:,1:] * learningRate) / m
    
    grad = np.concatenate((np.ravel(delta1),np.ravel(delta2)))
    np.ravel(delta1)
    
    return J,grad

In [53]:
J,grad = background_propagation(params,input_size,hidden_size,num_labels,X,y_onehot,learningRate)           
J,grad.shape

(6.799331755628472, (10285,))

In [55]:
from scipy.optimize import minimize

fmin  = minimize(fun=background_propagation,
                 x0=params,
                 args=(input_size, hidden_size, num_labels, X, y_onehot, learningRate),
                 method='TNC',jac=True,options={'maxiter':250})
fmin

     fun: 0.3426288258922007
     jac: array([-4.31748844e-04,  1.43958117e-06, -1.69905479e-06, ...,
        4.62757165e-04,  2.27110302e-04,  4.55018223e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 19
  status: 3
 success: False
       x: array([ 0.03088616,  0.00719791, -0.00849527, ...,  0.17615089,
        1.96005331, -0.39659801])

In [74]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size +1)],(hidden_size,(input_size + 1 ))))         
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size +1):],(num_labels,(hidden_size + 1 ))))

a1 ,z2 ,a2 , z3 ,h = forward_propagate(X,theta1,theta2)
y_pred = np.array(np.argmax(h,axis=1)+1)

In [75]:
correct = [1 if a == b else 0 for (a,b) in zip(y_pred,y)]
accuracy = (sum(map(int,correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 99.14%
